In [1]:
from dataset import create_dataset, create_sampler, create_loader
# from models.model_nlvr import XVLM
from models.model_pretrain import XVLM
from models import XVLMBase, build_mlp, load_pretrained

In [2]:
#import torch.distributed as dist
#dist.init_process_group('gloo', rank=0, world_size=1)

In [2]:
config = {'train_file': ['data/finetune/cosmos_caption_train.json'], 'val_file': ['data/finetune/cosmos_caption_val.json'], 
'test_file': ['data/finetune/cosmos_caption_val.json'], 
'image_root': 'images/cosmos/', 'vision_config': 'configs/config_swinB_384.json', 
'use_clip_vit': False, 'use_swin': True, 'image_res': 384, 'patch_size': 32, 
'use_roberta': False, 'text_config': 'configs/config_bert.json', 
'text_encoder': 'data/bert-base-uncased', 'batch_size_train': 20, 
'batch_size_test': 12, 'batch_size_test_text': 64, 'max_tokens': 40, 'embed_dim': 256, 
'temp': 0.07, 'k_test': 256, 'optimizer': {'opt': 'adamW', 'lr': 3e-05, 'weight_decay': 0.01, 
'lr_mult': 2}, 'schedular': {'sched': 'linear', 'lr': 3e-05, 'epochs': 10, 'num_warmup_steps': 0.1},
'prompt':''}

In [4]:
model = XVLM(config=config)
model.load_pretrained('checkpoint/4m_base_model_state_step_199999.th', config, is_eval=True)
model = model.to('cuda');

Position interpolate layers.0.blocks.0.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.0.blocks.1.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.1.blocks.0.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.1.blocks.1.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.2.blocks.0.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.2.blocks.1.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.2.blocks.2.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.2.blocks.3.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.2.blocks.4.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.2.blocks.5.attn.relative_position_bias_table from 13x13 to 23x23
Position interpolate layers.2.blocks.6.attn.relative_position_bias_tab

OSError: You seem to have cloned a repository without having git-lfs installed. Please install git-lfs and run `git lfs install` followed by `git lfs pull` in the folder you cloned.

In [5]:
train_dataset, val_dataset, test_dataset = create_dataset('caption_cosmos', config);

/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:834: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [6]:
datasets=[train_dataset, val_dataset, test_dataset]

In [7]:
train_loader, val_loader, test_loader = create_loader(datasets, [None, None, None], batch_size=[5,5,5],
                                                          num_workers=[4, 4, 4], is_trains=[True, False, False],
                                                          collate_fns=[None, None, None])

In [8]:
from models.tokenization_bert import BertTokenizer
from tqdm import tqdm
tokenizer = BertTokenizer.from_pretrained(config['text_encoder'])

In [9]:
import torch

In [11]:
image,text,_=train_dataset[500]
idx = torch.tensor(1)
image = image[None,:].to('cuda', non_blocking=True)
idx = idx.to('cuda', non_blocking=True)
text_input = tokenizer(text, padding='longest', max_length=config['max_tokens'], return_tensors="pt").to('cuda')

loss_itc, loss_itm = model(image, text_input.input_ids, text_input.attention_mask, idx=idx)
loss = loss_itc + loss_itm

zz
tensor(-0., device='cuda:0', grad_fn=<DivBackward0>)


RuntimeError: invalid multinomial distribution (sum of probabilities <= 0)